In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from utils import human_format

In [ ]:
file = corpusPath+'first_level_graph.tsv'

In [ ]:
df = pd.read_csv(file, sep='\t')

In [ ]:
print('Tweets:', human_format(len(df['tweet'].unique())))
print('URLs:', human_format(len(df['out_url'])))
print('Tweets without URL:', human_format(len(df['out_url'][df['out_url'].isin(['http://TweetWithoutURL.org'])])))
print('HTTP Errors:', human_format(len(df['out_url'][df['out_url'].isin(['http://HTTPError.org'])])))
print('Timeout Errors:', human_format(len(df['out_url'][df['out_url'].isin(['http://TimeoutError.org'])])))
print('Unique URLs:', human_format(len(df['out_url'][~df['out_url'].isin(['http://TweetWithoutURL.org', 'http://HTTPError.org', 'http://TimeoutError.org'])].unique())))

In [ ]:
#Keep only year
df['date'] = df['date'].apply(lambda s : datetime.strptime(s, '%d.%m.%y %H:%M').year)

In [ ]:
df1=df.copy()
df1['out_url'] = df1['out_url'].apply(lambda u: u if u in ['http://TweetWithoutURL.org', 'http://HTTPError.org', 'http://TimeoutError.org'] else 'http://WorkingURL.org')
df1[['tweet', 'date','out_url']].pivot_table(index='date', columns='out_url',aggfunc='count').T.reset_index(level=0, drop=True).T.fillna(1).plot(logy=True)

In [ ]:
df = df[~df['out_url'].isin(['http://TweetWithoutURL.org', 'http://HTTPError.org', 'http://TimeoutError.org'])]

df['netloc'] = df.apply(lambda r: '{0.netloc}/'.format(urlsplit(r['out_url'])), axis=1)
print('Unique network locations:', human_format(len(df['netloc'].unique())))
print('Unique government URLs:', human_format(len(df['netloc'][df['netloc'].str.contains('.gov')].unique())))

In [ ]:
inst = pd.read_csv(institutionsFile, sep='\t')
inst['URL'] = inst['URL'].apply(lambda u: re.sub(r'^(www[0-9]?\.)|(web\.)', r'', u))
def find_inst(netloc, inst):
    for i in inst:
        if i in netloc:
            return i
    return ''

df['netloc'] = df['netloc'].apply(lambda r: find_inst(r, inst['URL']))

In [ ]:
df = df.merge(inst, left_on='netloc', right_on='URL')

In [ ]:
print('Most popular Institutions')
df.groupby('Institution').mean()['popularity'].sort_values(ascending=False)[:10]

In [ ]:
df.groupby('Institution').mean().plot.scatter(x='World Rank', y='popularity')